In [14]:
from src.data.dataloaders import GMC_DataLoader, NIH_DataLoader
from src.models.train import train
from src.models.evaluate import test
from src.models.CheXNet import DenseNet121
from src.models.baseline import Baseline
from src.models.unet import UNet
from src.models.style_encoder import ConvEncoder
from src.models.super_model import SuperModel
from src.models.resnet import get_resnet

from torch import optim
import torch

%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
train_loader_GMC = GMC_DataLoader(stage='train', batch_size = 5, shuffle=True, num_workers=0, pin_memory=True,limit=1000)
train_loader_NIH= NIH_DataLoader(stage='train', batch_size = 5, shuffle=True, num_workers=0, pin_memory=True,limit=1000)
train_loaders = [train_loader_GMC, train_loader_NIH]

test_loader_GMC = GMC_DataLoader(stage='test', batch_size = 5, shuffle=True, num_workers=0, pin_memory=True,limit=1000)
test_loader_NIH= NIH_DataLoader(stage='test', batch_size = 5, shuffle=True, num_workers=0, pin_memory=True,limit=1000)
test_loaders = [test_loader_GMC, test_loader_NIH]


In [16]:
len(train_loader_NIH)

200

In [17]:
super_model = SuperModel()
optimizer = optim.Adam(super_model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

In [18]:
n_epochs = 20
irm = 0
vrex = 0

train(super_model, train_loaders, test_loaders, optimizer, criterion, irm, vrex, n_epochs, device, exp2 = True)

  0%|          | 0/4000 [00:00<?, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 32, 1, 1])

In [ ]:
from src.data.dataloaders import ENV_DataLoader

test_loader = ENV_DataLoader(batch_size=5,
                            shuffle=True, 
                            num_workers=0, 
                            pin_memory=True, 
                            stage = "test")
name_model = f'{super_model.name}_irm_{irm}_ep_{n_epochs}'
test(super_model, test_loader, name_model, device, exp2 = True)

  0%|          | 0/104 [00:00<?, ?it/s]

Accuracy of the model: 52.40384615384615 %


In [ ]:
name_model

'supermodel_irm_0_ep_20'